# **0. Test Execution Environment**

In [ ]:
!python --version

Python Version is 3.7.

In [ ]:
!nvidia-smi

If you fail plz change your run time to GPU in edit section -> note configure

# **1. INIT**

## 1-1. Mount Google Drive

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ln -s /content/gdrive/My\ Drive/ /mydrive
!mkdir /mydrive/yolov4
%cd /mydrive/yolov4

## 1-2. Yolo v4 install (Just FIRST INIT)

install in to your drive so run First Time

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
%cd darknet/
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

## 1-3. OpenCV

To use yolov4, OpenCV >= 4.4.0

In [ ]:
pip install --upgrade opencv-python

### OPTION!!

- Reference Link
    - [[1] Colab OpenCV With GPU](https://ichi.pro/ko/colab-eseo-gpuwa-hamkke-opencvleul-sayonghaneun-bangbeob-eun-mueos-ibnikka-41065314358367)  
    - [[2] Colab OpenCV WIth GPU](https://hanryang1125.tistory.com/18)

Difference is Python Version (2021/07 Python Version is 3.7)  
if compile work success, result is **cv2.cpython-37m-x86_64-linux-gnu.so**

#### ONLY INIT ONE TIME!!

It takes many time (2 hours over)  
So Be Patient!  

Copy result to your drive so run only **First Time**

In [ ]:
%cd /content
!git clone https://github.com/opencv/opencv
!git clone https://github.com/opencv/opencv_contrib
!mkdir /content/build
%cd /content/build
!cmake -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -DBUILD_SHARED_LIBS=OFF  -DBUILD_TESTS=OFF  -DBUILD_PERF_TESTS=OFF -DBUILD_EXAMPLES=OFF -DWITH_OPENEXR=OFF -DWITH_CUDA=ON -DWITH_CUBLAS=ON -DWITH_CUDNN=ON -DOPENCV_DNN_CUDA=ON /content/opencv
!make -j8 install

In [ ]:
!mkdir /mydrive/yolov4/cv2_gpu/python3
!cp  /content/build/lib/python3/cv2.cpython-37m-x86_64-linux-gnu.so   "/mydrive/yolov4/cv2_gpu/python3"

#### USAGE

In [ ]:
!pip uninstall opencv-python -y

Found existing installation: opencv-python 4.1.2.30
Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
!cp /mydrive/yolov4/cv2_gpu/cv2.cpython-37m-x86_64-linux-gnu.so /usr/local/lib/python3.7/dist-packages/

**should reload runtime**

#### Check

In [ ]:
import cv2

print(cv2.__version__)
print(cv2.cuda.getCudaEnabledDeviceCount())

cv2 Version: 4.5.3-dev  
cuda.getCudaEnableDeviceCount: 1  

Working good

## 1-4. Fast API & Ngrok Install

In [ ]:
pip install fastapi uvicorn pyngrok nest-asyncio > /dev/null 2>&1

# **2. OpenCV + YOLO v4**

[My Code from here](https://hanryang1125.tistory.com/9)

Click Run Button if error occur in FAST API

### MAIN

In [ ]:
import cv2
import numpy as np
import asyncio
import urllib.request

classes = []
with open("/mydrive/yolov4/darknet/cfg/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

def yolo(frame, size, score_threshold, nms_threshold):
    net = cv2.dnn.readNetFromDarknet("/mydrive/yolov4/darknet/cfg/yolov4.cfg", "/mydrive/yolov4/darknet/yolov4.weights")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # 이미지의 높이, 너비, 채널 받아오기
    height, width, channels = frame.shape

    # 네트워크에 넣기 위한 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (size, size), (0, 0, 0), True, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(blob)

    # 결과 받아오기
    outs = net.forward(output_layers)

    # 각각의 데이터를 저장할 빈 리스트
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.1:
                # 탐지된 객체의 너비, 높이 및 중앙 좌표값 찾기
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 객체의 사각형 테두리 중 좌상단 좌표값 찾기
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non Maximum Suppression (겹쳐있는 박스 중 confidence 가 가장 높은 박스를 선택)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=score_threshold, nms_threshold=nms_threshold)
    
    for idx in indexes:
        class_name = classes[class_ids[idx[0]]]
        if(class_name == "cat"): return True
    return False

def areYouCat(src=None):
    # 이미지 경로
    if(src == None): return False
    src = src[:-1]

    if ".jpg" not in src: 
        return False

    # 이미지 읽어오기
    url_response = urllib.request.urlopen(src)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)

    # 입력 사이즈 리스트 (Yolo 에서 사용되는 네크워크 입력 이미지 사이즈)
    size_list = [320, 416, 608]

    ret = yolo(frame=img, size=size_list[1], score_threshold=0.4, nms_threshold=0.4)
    return ret

### Async MAIN (Not Clear..)

In [ ]:
import cv2
import numpy as np
import asyncio
import urllib.request

classes = []
with open("/mydrive/yolov4/darknet/cfg/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

async def yolo(frame, size, score_threshold, nms_threshold):
    net = cv2.dnn.readNetFromDarknet("/mydrive/yolov4/darknet/cfg/yolov4.cfg", "/mydrive/yolov4/darknet/yolov4.weights")
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    # 이미지의 높이, 너비, 채널 받아오기
    height, width, channels = frame.shape

    # 네트워크에 넣기 위한 전처리
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (size, size), (0, 0, 0), True, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(blob)

    # 결과 받아오기
    outs = net.forward(output_layers)

    # 각각의 데이터를 저장할 빈 리스트
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.1:
                # 탐지된 객체의 너비, 높이 및 중앙 좌표값 찾기
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # 객체의 사각형 테두리 중 좌상단 좌표값 찾기
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non Maximum Suppression (겹쳐있는 박스 중 confidence 가 가장 높은 박스를 선택)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=score_threshold, nms_threshold=nms_threshold)
    
    for idx in indexes:
        class_name = classes[class_ids[idx[0]]]
        if(class_name == "cat"): return True
    return False

async def areYouCat(src=None):
    # 이미지 경로
    if(src == None): return False
    src = src[:-1]

    if ".jpg" not in src: 
        return False

    # 이미지 읽어오기
    url_response = urllib.request.urlopen(src)
    img_array = np.array(bytearray(url_response.read()), dtype=np.uint8)
    img = cv2.imdecode(img_array, -1)

    # 입력 사이즈 리스트 (Yolo 에서 사용되는 네크워크 입력 이미지 사이즈)
    size_list = [320, 416, 608]

    task = asyncio.create_task(yolo(frame=img, size=size_list[1], score_threshold=0.4, nms_threshold=0.4))
    ret = await task
    return ret

# **3. FAST API**

### FastAPI

In [ ]:

from urllib.parse import unquote
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import asyncio

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root(request: Request):
    url = str(request.query_params)
    
    if not url: return {'cat': 'false'}
    url = unquote(url)

    ret = areYouCat(url)
    if(ret == True): return {'cat': 'true'}
    return {'cat': 'false'}

import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.set_auth_token("YOUT AUTH TOKEN")
# your acccess token should be here
# sign in to ngrok

ngrok_tunnel = ngrok.connect(8000, bind_tls=True)

print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000, limit_concurrency=25)

### Async FastAPI (Not Clear..)

In [ ]:

from urllib.parse import unquote
from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import asyncio

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root(request: Request):
    url = str(request.query_params)
    
    if not url: return {'cat': 'false'}
    url = unquote(url)

    task = asyncio.create_task(areYouCat(url))
    ret = await task
    if(ret == True): return {'cat': 'true'}
    return {'cat': 'false'}

import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok.set_auth_token("YOUT AUTH TOKEN")
# your acccess token should be here
# sign in to ngrok

ngrok_tunnel = ngrok.connect(8000, bind_tls=True)

print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000, limit_concurrency=25)

# **4. View Section**

## 1. index.html

Using Socket.IO

In [ ]:
<!DOCTYPE html>
<html>
  <head>
    <title>Cat</title>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  </head>
  <body></body>
  
  <script src="https://cdn.socket.io/4.0.1/socket.io.min.js"></script>
  <script>
    const socket = io();
    socket.on('tweet', function(tweet){
      let target = document.querySelector("body");
      let p = document.createElement('img');
      
      p.setAttribute("src", tweet);
      target.appendChild(p);
    });
  </script>
</html>

## 2. Server

Using Socket.IO, Express, Node-Tweet-Stream, Node-Fetch  

Usage is localhost:8080 that's it!

In [ ]:
npm install socket.io express node-tweet-stream node-fetch --save

In [1]:
const app = require('express')();
const http = require('http').Server(app);
const io = require('socket.io')(http);
const port = process.env.PORT || 8080;

app.get('/', function(req, res){
  res.sendFile(__dirname + '/index.html');
});

const fetch = require('node-fetch');
const run = async (src) => {
  let url = "NGROK PUBLIC_URL/?";
  // plz input your public_url/?
  // YOU MUST ADD /? TO END OF URL 
  url += src;
  const response = await fetch(url);
  const json = await response.json();
  
  if(json['cat'] === 'false') return false;

  return true;
}

const Twitter = require('node-tweet-stream');
const twitter = new Twitter({
  consumer_key: "TWITTER API KEY",
  consumer_secret: "TWITTER API KEY",
  token: "TWITTER API KEY",
  token_secret: "TWITTER API KEY"
});

twitter.track('cat');

//twitter.track('고양이');
//twitter.language('ko');

//twitter.track('ねこ');
//twitter.track('ネコ');
//twitter.track('猫');
//twitter.language('ja');

twitter.on('tweet', tweet => {
  if(tweet.extended_entities === undefined) return;
  let medias = tweet.extended_entities.media;

  medias.forEach(media => {
    if(media.type === 'photo') 
    {
      run(media.media_url).then((ret => {
        if(true === ret) 
          io.emit('tweet', media.media_url);
      })).catch((err)=>{console.log(err)})
    }
  });
});

twitter.on('error', err => {
  console.error(err);
});

http.listen(port, function(){
  console.log('listening on *:' + port);
});

SyntaxError: ignored